In [1]:
import pandas as pd

In [60]:
# import Giant Bomb
gb = pd.read_csv("giantbomb/giantbomb_reviews.csv")

In [5]:
# import IGN
colHeader = ['game','date','reviewer','link','review']
IGN = pd.read_csv('IGN_1.csv', names = colHeader)
IGN['site'] = 'IGN'

In [36]:
# import Game Spot
colHeader2 = ['reviewer', 'date', 'game', 'link', 'review']
gamespot = pd.read_csv('gamespot.csv', names = colHeader2)
gamespot['site'] = 'GameSpot'

,reviewer,date,game,link,review,site
0,Justin Clark,8-May-15,Kerbal Space Program Review - GameSpot,http://www.gamespot.com/reviews/kerbal-space-p...,['Two astronauts are dead. And they represent ...,GameSpot
1,Brett Todd,8-May-15,Vertiginous Golf Review - GameSpot,http://www.gamespot.com/reviews/vertiginous-go...,"[""Build your vocabulary and your putting skill...",GameSpot
2,Britton Peele,8-May-15,Lost Orbit Review - GameSpot,http://www.gamespot.com/reviews/lost-orbit-rev...,['Youd think that being alone and adrift in ou...,GameSpot
3,Cameron Woolsey,8-May-15,Verdun Review - GameSpot,http://www.gamespot.com/reviews/verdun-review/...,"['The Battle of Verdun began in February 1916,...",GameSpot
4,Justin Haywald,7-May-15,BOXBOY! Review - GameSpot,http://www.gamespot.com/reviews/boxboy-review/...,"[""BoxBoy! is a game that subverts your expecta...",GameSpot


In [56]:
# convert date columns to datetime
gamespot["date"] = pd.to_datetime(gamespot.date, dayfirst=True)
IGN["date"] = pd.to_datetime(IGN.date, dayfirst=True)
gb["date"] = pd.to_datetime(gb.date, dayfirst=False)

In [57]:
results = pd.concat([gb, IGN, gamespot], ignore_index=True)

In [58]:
results.to_csv('results.csv', encoding='utf-8', index=False)

In [64]:
results.shape

(1361, 9)

In [66]:
results.mean()

score         3.525038
score_100    70.500759
dtype: float64